In [1]:
import pandas as pd
import wrangle
import new_wrangle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import model as m

import math
from sklearn.metrics import mean_squared_error, explained_variance_score

import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor 
from sklearn.model_selection import cross_val_score


# Wrangle

In [2]:
df =new_wrangle.wrangle_walmart()

# split and scale before modeling

In [3]:
train, test,  X_train_scaled, X_test_scaled, y_train, y_test = new_wrangle.split_scale(df, 'next_week_sales_target', MinMaxScaler())

train -> (2866, 32)
test -> (1229, 32)


# MODEL

In [4]:
# #conver y_train y _ validate to df
y_train_df = pd.DataFrame( {'actual': y_train})
y_test_df = pd.DataFrame( {'actual': y_test})

##  - BASELINE

In [5]:
#baseline version using last years sales
y_train_df['last_year_baseline'] = train['next_week_1_year_ago']

In [6]:
#calculate RMSE for baseline model
rmse_baseline2_train= math.sqrt(mean_squared_error(y_train_df.actual, y_train_df.last_year_baseline))

In [7]:
rmse_baseline2_train

91145.28223498359

In [8]:
#add the metric to our df
metric_df = pd.DataFrame(data = [{
    'model': 'baseline(using last year sales)',
    'rmse_train':rmse_baseline2_train,    
    'r^2' : 0}])

## MODEL: LinearRegression (OLS)

### - Gridsearch

In [9]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {'normalize': [ True, False],
          'fit_intercept': [True, False]}

In [10]:
#use a gridsearch function using 
m.gridsearch (X_train_scaled, y_train, LinearRegression() , params, 'neg_root_mean_squared_error')

,fit_intercept,normalize,score
2,False,True,-72272.953552
3,False,False,-72272.953552
1,True,False,-70476.190410
0,True,True,-70476.190410


### OLS uising select K best (6)

In [11]:
#using my function for SelectkBest
top_sb =m.select_kbest(X_train_scaled, y_train, 10)

The top 10 selected feautures based on the SelectKBest class are: ['this_week_sales', 'CPI', 'this_week_unemployment', 'store_size', 'next_week_1_year_ago', 'pre_christmas', 'cpi_4wk_rolling', 'unemp_4wk_rolling', 'cpi_quarterly_rolling', 'unemp_quarterly_rolling']


In [12]:
#create the model
ols_sb = m.create_model(X_train_scaled[top_sb], y_train_df, 'actual',\
                       LinearRegression(normalize=True,\
                        fit_intercept=True ), 'modelOLS' )
ols_sb['rmse']

73278.63464117885

#### cross validation

In [13]:
#create the model
clf = LinearRegression(normalize=True, fit_intercept=True )
#cv = number of folds
cross_val_score(clf, X_train_scaled[top_sb], y_train, cv=3, scoring = 'neg_root_mean_squared_error')

array([-73828.51208786, -73408.20028783, -74244.87809008])

### OLS uising  RFE (8)

In [14]:
#uise the fucntion to get RFE
top_rfe = m.select_rfe(X_train_scaled, y_train, 10,LinearRegression(normalize=True, fit_intercept=True ))

The top 10 selected feautures based on the the RFE class class are: ['this_week_sales', 'CPI', 'this_week_unemployment', 'next_week_1_year_ago', 'christmas', 'thanksgiving', 'cpi_4wk_rolling', 'avgMoM_perc_unemp', 'cpi_quarterly_rolling', 'unemp_quarterly_rolling']
this_week_sales             1
unemp_quarterly_rolling     1
cpi_quarterly_rolling       1
avgMoM_perc_unemp           1
CPI                         1
this_week_unemployment      1
cpi_4wk_rolling             1
next_week_1_year_ago        1
thanksgiving                1
christmas                   1
avgQoQ_perc_unemp           2
store_size                  3
fuel_quarterly_rolling      4
avgMoM_perc_fuel            5
fuel_4wk_rolling            6
avgQoQ_perc_fuel            7
this_week_holiday_flag      8
fuel_price                  9
unemp_4wk_rolling          10
avgMoM_perc_cpi            11
labor_day                  12
super_bowl                 13
temperature                14
avgQoQ_perc_cpi            15
pre_christmas 

In [15]:
#create the model
ols_rfe = m.create_model(X_train_scaled[top_rfe], y_train_df, 'actual', LinearRegression(normalize=True, fit_intercept=True ), 'modelOLS' )
ols_rfe['rmse']

71859.75745004068

#### cross validation

In [16]:
#create the model
clf = LinearRegression(normalize=True, fit_intercept=True )
#cv = number of folds
cross_val_score(clf, X_train_scaled[top_rfe], y_train, cv=3, scoring = 'neg_root_mean_squared_error')

array([-72127.07288044, -73241.00581704, -72267.9117694 ])

In [17]:
#add the best model  metrics to our metric_df
metric_df = metric_df.append(
    {
    'model': 'ols_rfe',
    'rmse_train': ols_rfe['rmse'],    
    'r^2' : ols_rfe['r2']}, ignore_index=True)

## Model :LassoLars

### - GridsearchCV

In [18]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'normalize': [True, False],
          'fit_intercept':[True, False],
           'alpha': [1.0, 0]
         }

In [19]:
#use the function to get the combinations of parameters
m.gridsearch (X_train_scaled, y_train, LassoLars() , params, 'neg_root_mean_squared_error')

,alpha,fit_intercept,normalize,score
2,1.0,False,True,-72298.372878
3,1.0,False,False,-72298.372878
6,0.0,False,True,-72272.953552
7,0.0,False,False,-72272.953552
5,0.0,True,False,-70476.190410
4,0.0,True,True,-70476.190410
0,1.0,True,True,-70458.146015
1,1.0,True,False,-70440.674321


### LassoLars using RFE 

In [20]:
#get the 6 features
top_rfe = m.select_rfe(X_train_scaled, y_train, 6, LassoLars(alpha = 1, normalize= False, fit_intercept= True) )

The top 6 selected feautures based on the the RFE class class are: ['this_week_sales', 'this_week_unemployment', 'next_week_1_year_ago', 'christmas', 'thanksgiving', 'unemp_quarterly_rolling']
this_week_sales             1
unemp_quarterly_rolling     1
thanksgiving                1
christmas                   1
this_week_unemployment      1
next_week_1_year_ago        1
avgMoM_perc_unemp           2
avgQoQ_perc_unemp           3
store_size                  4
avgMoM_perc_fuel            5
fuel_quarterly_rolling      6
fuel_4wk_rolling            7
avgQoQ_perc_fuel            8
this_week_holiday_flag      9
avgMoM_perc_cpi            10
fuel_price                 11
next_week_holiday_flag     12
temperature                13
CPI                        14
avgQoQ_perc_cpi            15
pre_christmas              16
super_bowl                 17
cpi_4wk_rolling            18
unemp_4wk_rolling          19
labor_day                  20
cpi_quarterly_rolling      21
dtype: int64


In [21]:
#create the model
lasso_rfe = m.create_model(X_train_scaled[top_rfe], y_train_df, 'actual', LassoLars(alpha = 1, normalize= False, fit_intercept= True), 'modelLasso' )
lasso_rfe['rmse']

72080.26921283794

In [22]:
### cross validation

In [23]:
#create the model
clf = LassoLars(alpha = 1, normalize= False, fit_intercept= True)
#cv = number of folds
cross = cross_val_score(clf, X_train_scaled[top_rfe], y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

array([-72369.74422049, -73054.01907945, -72590.01601532])

In [24]:
### LassoLars using selectKbest

In [25]:
#using my function for SelectkBest
top_sb =m.select_kbest(X_train_scaled, y_train, 10)

The top 10 selected feautures based on the SelectKBest class are: ['this_week_sales', 'CPI', 'this_week_unemployment', 'store_size', 'next_week_1_year_ago', 'pre_christmas', 'cpi_4wk_rolling', 'unemp_4wk_rolling', 'cpi_quarterly_rolling', 'unemp_quarterly_rolling']


In [26]:
lasso_skb = m.create_model(X_train_scaled[top_sb], y_train_df, 'actual', LassoLars(alpha = 1, normalize= False, fit_intercept= True), 'modelLasso' )
lasso_skb['rmse']

73281.87552494287

### cross validation

In [27]:
#create the model
clf = LassoLars(alpha = 1, normalize= False, fit_intercept= True)
#cv = number of folds
cross= cross_val_score(clf, X_train_scaled[top_sb], y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

array([-73875.3247233 , -73345.36555372, -74270.0989156 ])

**Note: the best LassoLars model is lasso_rfe**

In [28]:
#add the best model and its metrics
metric_df = metric_df.append(
    {
    'model': 'lasso_rfe',
    'rmse_train': lasso_rfe['rmse'],    
    'r^2' : lasso_rfe['r2']}, ignore_index=True)

## TweedieRegressor (GLM)

### - Gridsearch CV

In [29]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'power': [0.0, 1],
           'fit_intercept' : [True , False],
          'warm_start': [True, False], 
           'alpha': [1.0, 0.0]
         }
#use the function
m.gridsearch (X_train_scaled, y_train,TweedieRegressor() , params, 'neg_root_mean_squared_error')

,alpha,fit_intercept,power,warm_start,score
6,1.0,False,1.0,True,-1.109649e+06
7,1.0,False,1.0,False,-1.109649e+06
14,0.0,False,1.0,True,-1.106117e+06
15,0.0,False,1.0,False,-1.106117e+06
4,1.0,False,0.0,True,-5.226843e+05
5,1.0,False,0.0,False,-5.226843e+05
0,1.0,True,0.0,True,-4.957444e+05
1,1.0,True,0.0,False,-4.957444e+05
2,1.0,True,1.0,True,-1.442832e+05
3,1.0,True,1.0,False,-1.442832e+05


### create GLM using RFE

In [30]:
#use function to get the top 6 RFE
top_rfe = m.select_rfe(X_train_scaled, y_train, 6, TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False) )

The top 6 selected feautures based on the the RFE class class are: ['this_week_sales', 'store_size', 'next_week_1_year_ago', 'christmas', 'thanksgiving', 'avgQoQ_perc_unemp']
this_week_sales             1
thanksgiving                1
christmas                   1
next_week_1_year_ago        1
store_size                  1
avgQoQ_perc_unemp           1
avgMoM_perc_fuel            2
fuel_quarterly_rolling      3
fuel_4wk_rolling            4
avgQoQ_perc_fuel            5
unemp_4wk_rolling           6
this_week_unemployment      7
this_week_holiday_flag      8
avgMoM_perc_cpi             9
unemp_quarterly_rolling    10
avgMoM_perc_unemp          11
labor_day                  12
super_bowl                 13
temperature                14
fuel_price                 15
avgQoQ_perc_cpi            16
cpi_quarterly_rolling      17
next_week_holiday_flag     18
CPI                        19
pre_christmas              20
cpi_4wk_rolling            21
dtype: int64


In [31]:
#create the model
gml_rfe = m.create_model(X_train_scaled[top_rfe], y_train_df, 'actual',TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False), 'modelgml' )
gml_rfe['rmse']

71522.6640373484

In [32]:
#### cross validation

In [33]:
#create the model
clf = TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,warm_start= False)
#cv = number of folds
cross = cross_val_score(clf, X_train_scaled[top_rfe], y_train, scoring = 'neg_root_mean_squared_error', cv=3)
cross

array([-71446.65517559, -72245.35839113, -72581.66333012])

### create GML using select kbest

In [34]:
#using my function for SelectkBest
top_sb =m.select_kbest(X_train_scaled, y_train, 6)

The top 6 selected feautures based on the SelectKBest class are: ['this_week_sales', 'this_week_unemployment', 'store_size', 'next_week_1_year_ago', 'unemp_4wk_rolling', 'unemp_quarterly_rolling']


In [35]:
#create a model
gml_skb = m.create_model(X_train_scaled[top_sb], y_train_df, 'actual',TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False), 'modelgml' )
gml_skb['rmse']

73447.13066759182

#### cross validation

In [36]:
#create the model
clf = TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,warm_start= False)
#cv = number of folds
cross = cross_val_score(clf, X_train_scaled[top_sb], y_train, scoring = 'neg_root_mean_squared_error',cv=3)
cross

array([-74054.54562594, -73200.00828644, -74368.20223621])

**Note the best model for GLM is gml_rfe**

In [37]:
#add the best model
metric_df = metric_df.append(
    {
    'model': 'gml_rfe',
    'rmse_train': gml_rfe['rmse'],    
    'r^2' : gml_rfe['r2']}, ignore_index=True)

# Polynomial Regression

###  - Polynomila Regression using select k best 

In [38]:
#using my function for SelectkBest
top_sb =m.select_kbest(X_train_scaled, y_train, 6)

The top 6 selected feautures based on the SelectKBest class are: ['this_week_sales', 'this_week_unemployment', 'store_size', 'next_week_1_year_ago', 'unemp_4wk_rolling', 'unemp_quarterly_rolling']


In [39]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree = 3) 

# fit and transform X_train_scaled
X_train_degree3 = pf.fit_transform(X_train_scaled[top_sb])

# transform X_validate_scaled & X_test_scaled
X_test_degree3 = pf.transform(X_test_scaled[top_sb])

### GridSearchCV

In [40]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'normalize': [True, False],
          'fit_intercept':[True, False],
         }

m.gridsearch (X_train_degree3, y_train, LinearRegression() , params, 'neg_root_mean_squared_error')

,fit_intercept,normalize,score
0,True,True,-68367.964689
1,True,False,-68367.964689
2,False,True,-68367.964689
3,False,False,-68367.964689


In [41]:
#create a model
pol_skb = m.create_model(X_train_degree3, y_train_df, 'actual',LinearRegression( normalize=False, fit_intercept = False ), 'pol3_skb' )
pol_skb['rmse']

64044.02037218975

##### cross validation

In [42]:
#create the model
clf = LinearRegression( normalize=False, fit_intercept = True )
#cv = number of folds
cross = cross_val_score(clf, X_train_degree3, y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

array([-66722.79374691, -70656.60660325, -72587.28808291])

In [43]:
metric_df = metric_df.append(
    {
    'model': 'pol3_skb',
    'rmse_train': pol_skb['rmse'],    
    'r^2' : pol_skb['r2']}, ignore_index=True)

### -Polynoliam regression using RFE

In [44]:
#get the top 7 features by RFE
top_rfe_pol = m.select_rfe(X_train_scaled, y_train, 7, LinearRegression(normalize=False, fit_intercept = False) )

The top 7 selected feautures based on the the RFE class class are: ['this_week_sales', 'CPI', 'next_week_1_year_ago', 'cpi_4wk_rolling', 'unemp_4wk_rolling', 'cpi_quarterly_rolling', 'unemp_quarterly_rolling']
this_week_sales             1
unemp_quarterly_rolling     1
cpi_quarterly_rolling       1
CPI                         1
next_week_1_year_ago        1
unemp_4wk_rolling           1
cpi_4wk_rolling             1
this_week_unemployment      2
avgMoM_perc_unemp           3
avgMoM_perc_cpi             4
christmas                   5
thanksgiving                6
fuel_price                  7
fuel_4wk_rolling            8
store_size                  9
avgQoQ_perc_unemp          10
this_week_holiday_flag     11
avgQoQ_perc_fuel           12
avgQoQ_perc_cpi            13
avgMoM_perc_fuel           14
temperature                15
super_bowl                 16
pre_christmas              17
fuel_quarterly_rolling     18
labor_day                  19
next_week_holiday_flag     20
dtype: int

In [45]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree = 3) 

# fit and transform X_train_scaled
X_train_degree3 = pf.fit_transform(X_train_scaled[top_rfe_pol])

# transform X_validate_scaled & X_test_scaled
X_test_degree3 = pf.transform(X_test_scaled[top_rfe_pol])

#### GridsearchCV

In [46]:
m.gridsearch (X_train_degree3, y_train, LinearRegression() , params, 'neg_root_mean_squared_error')

,fit_intercept,normalize,score
0,True,True,-65466.028958
2,False,True,-64896.793744
3,False,False,-64896.793744
1,True,False,-64896.793729


In [47]:
#create the model
pol_rfe = m.create_model(X_train_degree3, y_train_df, 'actual',LinearRegression( normalize=False, fit_intercept = True ), 'pol3_RFE' )
pol_rfe['rmse']

60488.51113347658

#### cross validation

In [48]:
#create the model
clf = LinearRegression( normalize=True, fit_intercept = True )
#cv = number of folds
cross = cross_val_score(clf, X_train_degree3, y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

array([-65122.10841691, -67271.31287954, -66561.22897066])

In [49]:
metric_df = metric_df.append(
    {
    'model': 'pol3_rfe',
    'rmse_train': pol_rfe['rmse'],    
    'r^2' : pol_rfe['r2']}, ignore_index=True)

### -Polynomial Regression Degree 1  with all features

In [50]:
# make the polynomial features to get a new set of features using the entire df
pf = PolynomialFeatures(degree = 1) 

# fit and transform X_train_scaled
X_train_degree1 = pf.fit_transform(X_train_scaled)

# transform X_validate_scaled & X_test_scaled
X_test_degree1 = pf.transform(X_test_scaled)

### GreidSearch CV

In [51]:
m.gridsearch (X_train_degree1, y_train, LinearRegression() , params, 'neg_root_mean_squared_error')

,fit_intercept,normalize,score
2,False,True,-70476.19041
3,False,False,-70476.19041
1,True,False,-70476.19041
0,True,True,-70476.19041


In [52]:
pol_dg1 = m.create_model(X_train_degree1, y_train_df, 'actual',LinearRegression( normalize=True, fit_intercept = True ), 'modelpol' )
pol_dg1['rmse']

69757.43755006164

### Cross Validation

In [53]:
#create the model
clf = LinearRegression( normalize=True, fit_intercept = True )
#cv = number of folds
cross = cross_val_score(clf, X_train_degree1, y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

array([-69529.70428164, -71656.45914894, -71583.06247297])

In [54]:
#add this model metrics to df
metric_df = metric_df.append(
    {
    'model': 'pol_dg1',
    'rmse_train': pol_dg1['rmse'],    
    'r^2' : pol_dg1['r2']}, ignore_index=True)

## RESULTS

In [55]:
metric_df.sort_values('rmse_train')

,model,rmse_train,r^2
5,pol3_rfe,60488.511133,0.988269
4,pol3_skb,64044.020372,0.986849
6,pol_dg1,69757.437550,0.984398
3,gml_rfe,71522.664037,0.983598
1,ols_rfe,71859.757450,0.983443
2,lasso_rfe,72080.269213,0.983309
0,baseline(using last year sales),91145.282235,0.000000


In [56]:
print('The best model is' )
metric_df.nsmallest(1, 'rmse_train')

The best model is


,model,rmse_train,r^2
5,pol3_rfe,60488.511133,0.988269


**Takeaways**

the best model is Polynomial Regression Degree 2 using RFE top 7 Features
- 'this_week_sales', 
- 'CPI', 
- 'next_week_1_year_ago', 
- 'cpi_4wk_rolling', 
- 'unemp_4wk_rolling', 
- 'cpi_quarterly_rolling', 
- 'unemp_quarterly_rolling'

# TEST

In [57]:
#use my function to create and calculate the metrics
pol_reg_test =  m.create_model(X_test_degree3, 
                              y_test_df, 'actual',LinearRegression( normalize=False, fit_intercept = True ), 'test_polreg' )

In [58]:
y_test_df.head(15)

,actual,test_polreg
id,,
2011-03-18_store_7_2011-03-25,559061.58,5.651159e+05
2012-10-19_store_41_2012-10-26,1316542.59,1.340907e+06
2012-02-17_store_36_2012-02-24,313270.45,3.572269e+05
2012-03-16_store_26_2012-03-23,874790.68,8.744982e+05
2012-03-02_store_43_2012-03-09,636677.67,6.353168e+05
2012-02-03_store_42_2012-02-10,674919.45,6.385061e+05
2011-11-25_store_41_2011-12-02,1292436.23,1.378522e+06
2012-02-17_store_23_2012-02-24,1272948.27,1.269420e+06
2011-04-08_store_23_2011-04-15,1263680.51,1.296745e+06


In [59]:
y_train_df.head()

,actual,last_year_baseline,modelOLS,modelLasso,modelgml,pol3_skb,pol3_RFE,modelpol
id,,,,,,,,
2011-09-16_store_12_2011-09-23,871692.74,851919.34,8.414476e+05,8.280014e+05,8.265554e+05,8.818270e+05,9.082096e+05,8.692667e+05
2012-08-10_store_4_2012-08-17,2283540.30,2232892.10,2.261697e+06,2.256597e+06,2.252690e+06,2.364183e+06,2.326712e+06,2.272793e+06
2011-12-09_store_27_2011-12-16,2205919.86,2350098.36,2.249371e+06,2.262273e+06,2.252397e+06,2.190291e+06,2.211375e+06,2.239841e+06
2011-12-30_store_1_2012-01-06,1550369.92,1444732.28,1.459655e+06,1.447282e+06,1.453858e+06,1.479411e+06,1.500979e+06,1.443146e+06
2012-08-31_store_43_2012-09-07,663814.18,649128.23,6.338107e+05,6.250764e+05,6.222088e+05,6.228781e+05,5.942555e+05,6.539501e+05
